# **Fundamentals of arbitrage and options**
*A gentle introduction to the fundamentals of arbitrage*
---
 <br>
 
 
 
- Copyright (c) Lukas Gonon, 2024. All rights reserved

- Author: Lukas Gonon <l.gonon@imperial.ac.uk>

- Platform: Tested on Windows 10 with Python 3.8

Disclaimer: All data used in this course are publicly available

In [1]:
from platform import python_version
print("Current python version: ", python_version())

Current python version:  3.11.1


## Session Overview

- Review: financial markets and traded assets and derivatives
- Review: a first look at arbitrage pricing (bounds on call option prices)
- Fundamental concept of arbitrage
- Pricing derivatives by replication

# ARBITRAGE or *how to evaluate the price of a contingent claim?*

## Modelling assumptions

- No market frictions (no transaction costs, no bid/ask spread, no taxes)
- No default risk (same interest for borrowing and lending)
- Competitive markets (Market participants act as price takers, not price makers)
- Rational agents (Market participants prefer more to less)
- No arbitrage

Financial markets involve riskless (bank account) and risky (stocks,....) assets. 
The only reason for the investor to expose herself to risk is the opportunity of greater profit.

## Creating an arbitrageable portfolio

Consider a portfolio consisting of a (risk-free) bond, a (risky) stock and a European Call option with strike $K$, in a one-time period setting:
$$
\Pi_t = n_B B_t + n_S S_t + n_C C_t,
\qquad\text{for } t \in \{0,T\}.
$$
At time $0$, the prices of the bond, the underlying stock and the Call are given by $B_0, S_0, C_0$.
At time $T=1$, the bond price has moved to $B_1$ while the stock price has either gone up to $S_1^u$ or down to $S_1^d$.
Therefore, at maturity, the payoff of the Call option reads
\begin{align*}
C_T^u = (S_T^u-K)_+ \qquad\text{or}\qquad C_T^d = (S_T^d-K)_+.
\end{align*}

In [2]:
import numpy as np

In [3]:
def portfolio_value(quantities, B, S, C, details):
    nB, nS, nC = quantities
    if details:
        print("Bond/Stock/Call amounts: ", 
              np.round(nB*B, 5), 
              np.round(nS*S, 5), 
              np.round(nC*C, 5))
    return np.round(nB*B + nS*S + nC*C, 5)

def callPayoff(S, K):
    return max(S-K, 0.)

In [4]:
### Initial values:
B0, S0, C0, K = 1., 1., .2, 1.

### Up and Down values for the stock price at time 1
Sd, Su = .75, 1.75

### Bond price at time 1
B1 = 1.25

Cd, Cu = callPayoff(Sd, K), callPayoff(Su, K)
print("At expiry, the Call price is worth %.2f in the Down state and %.2f in the Up state" %(Cd, Cu))

At expiry, the Call price is worth 0.00 in the Down state and 0.75 in the Up state


At maturity, the portfolio can take two different values, depending on the state (up or down).

In [5]:
### Quantities of the bond, stock and Call in the portfolio:
n_Bond1, n_Stock1, n_Call1 = 40, 40, 100

quantities_Ptf1 = [n_Bond1, n_Stock1, n_Call1]

print("Value of the portfolio at time zero:", portfolio_value(quantities_Ptf1, B0, S0, C0, False))

print("Value of the portfolio at expiry in the Down and the Up states:", 
      portfolio_value(quantities_Ptf1, B1, Sd, Cd, False), 
      portfolio_value(quantities_Ptf1, B1, Su, Cu, False))

Value of the portfolio at time zero: 100.0
Value of the portfolio at expiry in the Down and the Up states: 80.0 195.0


Let us now consider an alternative portfolio, i.e. on the same assets, but with different weights.

In [6]:
### Quantities of the bond, stock and Call in the new portfolio:
n_Bond2, n_Stock2, n_Call2 = 47.2, 28, 116
quantities_Ptf2 = n_Bond2, n_Stock2, n_Call2

print("New portfolio at time zero:", portfolio_value(quantities_Ptf2, B0, S0, C0, False))

print("New portfolio in down and up states at expiry:", 
      portfolio_value(quantities_Ptf2, B1, Sd, Cd, False),
      portfolio_value(quantities_Ptf2, B1, Su, Cu, False))

New portfolio at time zero: 98.4
New portfolio in down and up states at expiry: 80.0 195.0


**WHAT IS THE PROBLEM?**

Consider the difference of the two portfolios (itself a portfolio):
$$
D = P^{1} - P^{2}
$$

In [7]:
n_Bond = n_Bond1 - n_Bond2
n_Stock = n_Stock1 - n_Stock2
n_Call = n_Call1 - n_Call2

quantities = n_Bond, n_Stock, n_Call

print("Difference portfolio at time zero:", portfolio_value(quantities, B0, S0, C0, False))

print("Difference portfolio in the Down and Up states at expiry:", 
      portfolio_value(quantities, B1, Sd, Cd, False),
      portfolio_value(quantities, B1, Su, Cu, False))

print("Quantities: ", quantities)

Difference portfolio at time zero: 1.6
Difference portfolio in the Down and Up states at expiry: -0.0 -0.0
Quantities:  (-7.200000000000003, 12, -16)


*Question: What do you think?*

A closer look

In [8]:
print("Value of the difference portfolio 1 at time zero:", 
      portfolio_value(quantities, B0, S0, C0, True))

Bond/Stock/Call amounts:  -7.2 12.0 -3.2
Value of the difference portfolio 1 at time zero: 1.6


This corresponds to the position: sell stock, buy bond and option.

However, we can see that the position (sell stock, buy bond) is actually a perfect **hedge** for a long position in the Call option.

**--> Hedging and pricing by replication<--**

**No Arbitrage Definition: No profit is guaranteed without exposure to risk.**

## Pricing by replication (introducing probabilities)

- Fischer Black and Myron Scholes. <a href="https://www.journals.uchicago.edu/doi/10.1086/260062">"The Pricing of Options and Corporate Liabilities"</a>. Journal of Political Economy (1973). 
- Robert C. Merton. <a href="https://www.jstor.org/stable/3003143?origin=crossref">"Theory of Rational Option Pricing"</a>. Bell Journal of Economics and Management Science (1973).

Merton and Scholes got awarded the Nobel Memorial Prize in Economic Sciences "for a new method to determine the value of derivatives" in 1997.

**Meta-Theorem: Under no-arbitrage settings, two portfolios generating the same values in all states of the world should be equal at all times.**

*Question: If two quoted prices are different (say because of two different vendors on an OTC market), how can you benefit from it?*

Consider a bond with constant value equal to $1$ and a stock price with $S_0 = 10$ and
\begin{equation*}
\left\{
\begin{array}{ll}
S_1^u = 20, & \text{ with probability }p^u,\\
S_1^d = 7.5, & \text{ with probability }p^d,
\end{array}
\right.
\end{equation*}
for some $p^d,p^u \in [0,1]$ with $p^d + p^u = 1$.

**GOAL:** Determine the price of a European Call option on $S$ with maturity $T=1$ and strike $K=15$.

**STRATEGY**: Construct a portfolio $\Pi := n_B B + n_S S$ replicating exactly the option payoff.

Construct 

$$
D := \Pi - C = n_B B + n_S S + n_{C} C,
$$
with $n_C = -1$.

If $\Pi$ replicates $C$, i.e. $\Pi_T = C_T$ (in all states of the world),
then
$$
D_T = 0.
$$

Remarks:
- The vector $(n_B, n_S)$ is called a strategy.
- What about the probabilities $p^d$ and $p^u$? Where do they come from?

**First Fundamental Theorem of Asset Pricing (FTAP):
No-arbitrage is equivalent to the existence of an equivalent martingale
measure.**


In other words, under no-arbitrage, we can find probabilities 
$\tilde{p}^d, \tilde{p}^u \in [0,1]$ with $\tilde{p}^d + \tilde{p}^u = 1$ such that
$$
S_0 = \tilde{p}^d S_1^d + \tilde{p}^u S_1^u.
$$

## Summarising

Let $S$ denote a stock price, whose value at time $0$ is $S_0>0$.
At time $T>$, the process can take two possible values:
$$
S_1=
\left\{
\begin{array}{rll}
uS_0, & \text{with probability } p,\\
dS_0, & \text{with probability } 1-p,
\end{array}
\right.
$$
where $0<d<u$ and $p\in(0,1)$.

*Note: Another way to understand this is to write $S_1=X S_0$, where $X$ is a Bernoulli random variable taking the value $u$ with probability $p$ and the value $d$ with probability $1-p$.*

We assume that a financial agent can invest (at time $0$) in both the asset $S$ and in a risk-free bond, 
i.e. borrow or sell money with a (non random) interest rate equal to $r\geq 0$ over the period $[0,T]$.
We are now interested in determining the price $C_0$ at time $0$ of a European Call option with strike $K>0$ and maturity $T$.
At time $T$, the two possible payoffs (corresponding to the two different states of the world) are
\begin{align*}
C_1^{u} & :=\left(uS_0-K\right)_+,\\
C_1^{d} & :=\left(dS_0-K\right)_+.
\end{align*}

One may be tempted to value it at the price 
$$
C_0
 = \frac{\mathbb{E}_p[C_1]}{1+r}
 = \frac{pC_1^{u}+(1-p)C_1^{d}}{1+r}.
$$
However, this is in general false, since the probability $p$ has been chosen from the investor's point of view, 
and does not necessary reflect the market's point of view.
It turns out that this very probability $p$, called the historical (or physical) probability, does not appear at all in the pricing formula, 
as the following theorem shows.

**Theorem.
In the absence of arbitrage opportunities, the price at time $0$ (inception of the contract) of a European call option written on $S$, 
with strike $K$ and maturity $T$ is worth
$$
C_0=\frac{\pi C_1^{u}+(1-\pi) C_1^{d}}{1+r},
\qquad\text{where }\pi:= \frac{1+r-d}{u-d}.
$$**
The probability $\pi$ is called the risk-neutral probability.

#### Proof

The proof is again based on pricing by replication, 
i.e. we want to construct a portfolio consisting of shares and risk-free bonds 
that exactly replicates (has the same payoff as) the option we wish to evaluate.
Consider a portfolio $\Pi$ consisting of an amount $\Delta_0$ of shares and 
with the notional $\phi$ invested in the risk-free bond.
The value at time $t_0$ of the portfolio is therefore 
$$
\Pi_0=\Delta_0 S_0+\phi B_0.
$$
At time $t_1$, it is worth
$$
\Pi_1 = \Delta_0 S_1 + \left(1+r\right)\phi
 = 
\left\{
\begin{array}{rll}
\Delta_0 uS_0 + \left(1+r\right)\phi, & \text{with probability } p,\\
\Delta_0 dS_0 + \left(1+r\right)\phi, & \text{with probability } 1-p.
\end{array}
\right.
$$
Since our portfolio $\Pi$ has to replicate the option, it therefore needs to have the same payoff.
This implies the following system of equations:
$$
\left\{
\begin{array}{rl}
\Delta_0 uS_0 + (1+r)\phi & = C_1^{u} = (uS_0-K)_+\\
\Delta_0 dS_0 + (1+r)\phi & = C_1^{d} = (dS_0-K)_+,
\end{array}
\right.
$$
which we can solve explicitly as
$$
\Delta_0 = \frac{C_1^{u}-C_1^{d}}{(u-d)S_0},
\qquad\text{and}\qquad
\phi = \frac{1}{1+r}\frac{uC_1^{d}-dC_1^{u}}{(u-d)}.
$$
By absence of arbitrage, since our portfolio $\Pi$ and the Call option have the same value at maturity (same payoff), 
then they necessarily have the same value at inception of the contract, i.e. at time $t_0$, 
so that $C_0=\Pi_0$.
Define now $\pi:=\displaystyle \frac{1+r-d}{u-d}$ and the theorem follows.

*Exercises*
- *Application to previous example*
- *Verifying put-call parity*

In [9]:
u = Su/S0
r=B1/B0-1
d = Sd/S0
pi = (1+r-d)/(u-d)
C_= (pi*Cu+(1-pi)*Cd)/(1+r)
print(r)
print(d)
print(pi)
print(C_)

0.25
0.75
0.5
0.3


*What happens above if we use this price?*

# Wrapping up in a more general (abstract) setup

The fundamental model of mathematical finance consists of a probability space $\left(\Omega, \mathcal{F},\mathbb{P}\right)$
on which we define a random variable $S$.
The most obvious example is when $S$ takes values in $\mathbb{R}$ or $\mathbb{R}_+$
and represents the price of a stock (or an interest rate, the price of some commodity...) at some given time, 
or when it is $\mathbb{R}^n$-valued ($n\in\mathbb{N}$) and accounts for the basket of share prices in an index such as the S\&P$500$.
One can also think of it as an infinite-dimensional random variable representing the whole path of a share price process between today and some future time, i.e. $S=\left(S_t\right)_{t\geq 0}$.

A financial derivative (contingent claim) written on $S$ can then be thought of as a functional $f(S)$.
Financial derivatives are usually classified according to whether $S$ represents the value of the share price at some future time $T>0$ 
(European options) or the whole trajectory between today (time zero) and time $T$ (American options).
One of the fundamental questions in mathematical finance is to evaluate such functionals, 
i.e. to determine at time zero (inception  of the contract) the expected value of $f(S)$. 
Intuitively speaking we wish to answer the following question: 
$$
\text{How much are we willing to pay today (time zero) to receive $f(S)$ at time $T$?}
$$

The answer lies in the concept of *absence of arbitrage*.
Consider a portfolio---or a trading strategy---as a random process $(V_t^\theta)_{t\geq 0}$ consisting of some positions in
$n$ stock prices $S^{(1)},\ldots,S^{(n)}$:
$$
V_t^\theta=\sum_{i=1}^n \theta_t^{(i)}S_{t}^{(i)},\qquad \text{for all }t\geq 0,
$$
where $\theta_{t}^{(i)}$ represents the quantity of stock $i$ in the portfolio at time $t$.
We have written here $V_t^\theta$ to emphasise the fact that 
the strategy is fully determined by the (time-dependent) vector $\theta$.
In a discrete-time setting, let us fix some time $t>0$.
At time $t+1$, the investor may want to rebalance his portfolio, i.e. change its composition, and the value of the portfolio hence becomes
$$
V_{t+1}^\theta=\sum_{i=1}^n \theta_{t+1}^{(i)}S_{t+1}^{(i)}.
$$
If we assume that the investor does not invest nor withdraw any amount from his portfolio, then we necessarily have
$$
\sum_{i=1}^n \theta_{t}^{(i)}S_{t+1}^{(i)}=\sum_{i=1}^n \theta_{t+1}^{(i)}S_{t+1}^{(i)}.
$$
This can be written equivalently
\begin{align*}
V_{t+1}^\theta-V_{t}^\theta
 & = \sum_{i=1}^n \theta_{t+1}^{(i)}S_{t+1}^{(i)} - \sum_{i=1}^n \theta_{t}^{(i)}S_{t}^{(i)}\\
 & = \sum_{i=1}^n \theta_{t}^{(i)}S_{t+1}^{(i)} - \sum_{i=1}^n \theta_{t}^{(i)}S_{t}^{(i)}\\
 & = \sum_{i=1}^n \theta_{t}^{(i)}\left(S_{t+1}^{(i)}-S_{t}^{(i)}\right),
\end{align*}
and we shall call such a portfolio *self-financing*.

We shall further call a trading strategy *admissible* if it is self-financing and if $V_t^\theta\geq 0$ for all $t\geq 0$.

**Definition: An arbitrage is an admissible trading strategy (or a portfolio) $V^\theta=\left(V_t^\theta\right)_{t\geq 0}$ 
for which there exists some time $T>0$ such that
$$V_0^\theta=0 \text{ and }
V_T^\theta\geq 0 \text{ almost surely}
\qquad\text{and}\qquad
\mathbb{P}\left(V_T^\theta>0\right)>0.$$**

Intuitively the concept *absence of arbitrage* therefore means that one cannot make a sure profit out of nothing.
In this definition we have used a probability $\mathbb{P}$ given ad hoc as an element of the probability space.
However for practical and theoretical reasons---which shall be made clear later in this course---we might want to use other probabilities, 
which are *equivalent* (in some sense to be made precise).
In the discrete-time setting used above, we consider a family of random variables 
$X:=\left(X_{t_1},\ldots,X_{t_n},\ldots\right)$ indexed by time steps.
Consider further the family of nested sets $\left(\mathcal{F}_n\right)_{n\geq 1}$ satisfying 
$\mathcal{F}_n\subset\mathcal{F}_{n+1}\subset\mathcal{F}$ for any $n\geq 1$.
We shall informally--- and leave the rigorous definition for later---consider 
$\mathcal{F}_n$ as the quantity of information available at time $t_n$ generated by the random variables $X_{t_1},\ldots,X_{t_n}$.

**Definition: We say that the family of random variables $X=\left(X_{t_1},\ldots,X_{t_n},\ldots\right)$ is a martingale if the equality 
$\mathbb{E}[X_{t_n}|\mathcal{F}_{t_p}]=X_{t_p}$ holds for any $1\leq p\leq n$.**

*Example: If $\left(Y_n\right)_{n\geq 1}$ forms a family of independent identically distributed random variables 
such that for any $n\geq 1$, $\mathbb{E}[Y_n|\mathcal{F}_n]=0$ and 
$\left(\mathcal{F}_n\right)_{n\geq 1}$ is the related flow of information, then the family $(X_n)_{n\geq 1}$ defined by $X_n:=\sum_{i=1}^{n}Y_i$ is a martingale.*

**Definition: A probability measure $\mathbb{Q}$ is a martingale measure equivalent to $\mathbb{P}$, and
we denote it by $\mathbb{Q}\sim\mathbb{P}$ if discounted price processes are martingales under $\mathbb{Q}$.**

**Theorem [Fundamental Theorem of Asset Pricing]: A model is said to be arbitrage-free (i.e. there does not exist any admissible arbitrage strategy) if and only if 
there exists an equivalent martingale measure $\mathbb{Q}$.**

This fundamental theorem has the following immediate application in terms of pricing:
under absence of arbitrage, the price at time zero of a financial derivative is equal to the discounted expectation of the final payoff
$f(X)$ under the martingale measure $\mathbb{Q}$, namely
$$
\mathbb{E}^{\mathbb{Q}}[f(X)].
$$


*What does this mean in the model above?*

The martingale measure $\mathbb{Q}$ is determined by the risk-neutral probability $\pi$.  $\pi$ is the probability under  $\mathbb{Q}$ that $S_1 = u S_0$ and $1-\pi$ is the probability that $S_1 = d S_0$. 

Let us verify the martingale property. In this simple situation, $\mathbb{E}[X_{1}|\mathcal{F}_{0}]=\mathbb{E}[X_{1}]$ and so we need to verify $\mathbb{E}^\mathbb{Q}[S_{1}/(1+r)]=S_0$.

In [10]:
print(pi*Su+(1-pi)*Sd-(1+r)*S0)

0.0
